# Taller 3: Clasificación

## [Introducción a los Sistemas Inteligentes](https://dis.unal.edu.co/~fgonza/)


-------------------

Aima ha notado tu presencia en la Unión Nacional de Algoritmos de Localización (UNAL). Siendo esta una institución de alto prestigio ha decidido ponerte a prueba. Para ello te ha asignado a trabajar en un challenge de Kaggle.

El hundimiento del RMS Titanic es uno de los naufragios más famosos de la historia. El 15 de abril de 1912, el Titanic se hundió después de estrellarse con un iceberg, matando 1502 de 2224 pasageros y tripulación. Este evento sacudió toda la comunidad internacional e implico mejoras en las medidas de seguridad para Barcos.

Una de las razones de tantas perdidas humanas fue la falta de botes salvavidas. Aunque la sobrevivencia de una persona se regia un poco por la suerte algunos grupos de personas tenian mayor chance de sobrevivir que otros, como mujeres, niños y miembros de la clase alta.

En este reto se le solicita que complete el análisis sobre que tipo de personas eran más propensas a sobrevivir. En particular se le solicita que aplique las herramientas del machine learning para predecir que pasajeros del RMS Titanic sobrevivieron.
[Ver más](https://www.kaggle.com/c/titanic/overview)


**Para descargar el archivo de datos en el siguiente link: [titanic.csv](https://drive.google.com/file/d/1KZD9Ic2Gmd39yLlFlHSSs6lgDqyWuPCW/view?usp=sharing)**

----------------------------------
La siguiente tabla muestra información sobre (algunas) variables presentes en el dataset

Data Dictionary
```
Variable	Definition	Key
survived 	Survival 	0 = No, 1 = Yes
pclass		Ticket class 	1 = 1st, 2 = 2nd, 3 = 3rd
sex 		Sex 	
Age 		Age in years 	
sibsp 		# of siblings / spouses aboard the Titanic 	
parch 		# of parents / children aboard the Titanic 	
ticket 		Ticket number 	
fare 		Passenger fare 	
cabin 		Cabin number 	
embarked 	Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton
```

#### Variable Notes

- **pclass**: A proxy for socio-economic status (SES)
    1st = Upper
    2nd = Middle
    3rd = Lower

- **age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

- **sibsp**: The dataset defines family relations in this way...
    Sibling = brother, sister, stepbrother, stepsister
    Spouse = husband, wife (mistresses and fiancés were ignored)

- **parch**: The dataset defines family relations in this way...
    Parent = mother, father
    Child = daughter, son, stepdaughter, stepson

    Some children travelled only with a nanny, therefore parch=0 for them.

-------------------

#### Notas adicionales:

Sus modelos seran evaluados usando la métrica accuracy. No modifique la firma de las funciones (nombre y parametros). La plataforma de evaluación tendrá una copia del archivo titanic.csv. Para su evaluación local, deben descargar el archivo y subirlo a Colab.

# 1 Procesamiento y Exploración de Datos

## 1.1 Cargue los datos
* Algunas columnas tienen valores null, este es un reto común con el que se encontrar. Más adelante nos ocuparemos de esto. 
* Extraiga las features y el target del dataframe en dos variables. X, y.
* Antes de ejecutar el código en la siguiente celda cargue el archivo `titanic.csv` en la raiz del sistema de archivos del ambiente de ejecución usando la opción correspondiente en el menú lateral.

In [1]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

# Path donde se encuentra el archivo de datos.
path = 'titanic.csv'
df = pd.read_csv(path, index_col='PassengerId')

X, y = df.drop(axis=1,columns=['Survived']), df.Survived

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


## 1.2 Extraiga las variables númericas que no tengan datos faltantes del dataset
Debe determinar que variables columnas son de tipo numérico y no tienen datos faltantes. Para hacer esto use el atributo `dtypes` de los objetos de tipo `DataFrame` y el método `count()`.

Implemente la función `extract_numerial` para obtener estos features del Dataframe.

In [2]:
def extract_features(X, features):
    '''
    X: dataframe como se define en 1.1
    features: lista de features a ser extraidas
    returns: X derivado con únicamente las columnas en features
    '''
    return X[features]

def extract_numerical(X):
    '''
    X: dataframe como se define en 1.1
    returns: dataframe derivado que puede ser usado para entrenar un modelo (sin variables categoricas )
    '''
    nameV = list(X.dtypes.index)


    attr_ints = [] 
    numregs = df.shape[0] 

    for i in nameV:
      if (X.dtypes[i] in ["float64", "int64"]):
        if (X[i].count() == numregs):
          attr_ints.append(i)
    
    result = extract_features(X, attr_ints)

    return result

In [3]:
extract_numerical(df)

,Survived,Pclass,SibSp,Parch,Fare
PassengerId,,,,,
1,0,3,1,0,7.2500
2,1,1,1,0,71.2833
3,1,3,0,0,7.9250
4,1,1,1,0,53.1000
5,0,3,0,0,8.0500
...,...,...,...,...,...
887,0,2,0,0,13.0000
888,1,1,0,0,30.0000
889,0,3,1,2,23.4500


## 1.3 Entrene modelos de regresion logística y naïve bayes sobre los variables númericas

Para este punto debe implementar dos funciones:
- `train_logit`: para entrenar modelos de _Regresión Logística_. Esta función debe retornar un modelo ya entrenado de regresión logística siendo de la clase `LogisticRegression` de sklearn.
    El modelo logístico debe superar 0.68 de presición.


- `train_bayes`: para entrenar modelos de tipo _Naïve Bayes_. Esta función debe retornar un modelo ya entrenado de Naïve Bayes siendo de la clase `GaussianNB` de sklearn.
    Este modelo debe superar 0.67 de presición.
    
**Peso del punto: 2.0**

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

def train_logit(X, y):
    '''
    X: dataframe derivado de df como se define en 1.1
    y: target como se define en 1.1
    returns (LogisticRegression): modelo entrenado con los datos X y y
    '''
    #clf = clasificador
    clf = LogisticRegression()
    clf.fit(X, y)
    
    return clf

def train_bayes(X, y):
    '''
    X: dataframe derivado de df como se define en 1.1
    y: target como se define en 1.1
    returns (GaussianNB): modelo entrenado con los datos X y y
    '''
    clf2 = GaussianNB()
    clf2.fit(X, y)
    
    return clf2

In [5]:
# Cell for testing.
X_numerical = extract_numerical(X)
logit = train_logit(X_numerical, y)
bayes = train_bayes(X_numerical, y)

print(logit.score(X_numerical, y))
print(bayes.score(X_numerical, y))

0.6879910213243546
0.6778900112233446


## 1.4 Entrene los modelos con los datos procesados y la feature 'Sex' con label encoding
### 1.4.1 Haga label-encoding de la feature 'Sex'

Label encoding consiste en asignar un label a cada grupo de datos, en este caso 'female' obtiene el label 1 y 'male' el label 0, o al revés. Lo importante es que los labels sean valores númericos.

* Puede hacer un `for`
* Puede usar `.map()` de Pandas sobre `X['Sex']`
* Puede usar `sklearn.preprocessing.LabelEncoder`

Cree un DataFrame que contenga las característica numéricas del punto 1.2 y que además tenga una columna correspondiente a la característica 'Sex' con label encoding.
Para esto, implemente la función `process_data`.

**Peso del punto: 1.0**

In [6]:
def process_data(X):
    '''
    X : dataframe como se define en 1.1
    returns (Dataframe): dataframe derivado de X con columnas númericas incluyendo 'Sex' con label encoding
    '''
    serieSexo = extract_features(X, "Sex").map({"male":0,"female":1})
    numRegs = X.shape[0]    
    listSex = list (serieSexo.values)
    
    X_numerical = extract_numerical(X)
    X_numerical_ampliado = X_numerical.assign(Sex = listSex)
    
    return X_numerical_ampliado
    

### 1.4.2 Entrene los modelos usando la nueva feature 'Sex'
Usando las funciones ya implementadas anteriormente, haga pruebas con el nuevo Dataframe que contiene la columna 'sex'.


**El modelo logístico debe superar 0.79 de precisión y bayes 0.78.**

In [7]:
X_processed = process_data(X)
logit = train_logit(X_processed, y)
bayes = train_bayes(X_processed, y)

print(logit.score(X_processed, y))
print(bayes.score(X_processed, y))

0.7957351290684624
0.7890011223344556


# 2 Métricas de Desempeño

## 2.1 Defina una función que encuentre los Falsos Positivos y los Falsos Negativos de un modelo en un conjunto de prueba.

Para este punto debe considerar como negativo _no supervivencia_ y como positivo _supervivencia_.

Implemente la función `fpFn` que retorne una tupla de tipo (FP, FN) donde _FP_ son los falsos positivos y _FN_ los falsos negativos.

**Peso del punto: 1.0**

In [3]:
def fpFn(y_true, y_pred):
    '''
    y_true: lista con las etiquetas originales del dataset
    y_pred: lista con las etiquetas predichas por un modelo
    Returns:
    (FP, FN) : Tupla donde FP son los falsos positivos, FN son los falsos negativos
    '''
    FP = 0
    FN = 0

    casos = len(y_true)
    for i in range(casos):
      if (y_pred[i] == 1 and y_true[i] == 0):
        FP += 1
      if (y_pred[i] == 0 and y_true[i] == 1):
        FN += 1
    return (FP,FN)

## 2.2 (1.0) Defina una función que encuentre la Sensibilidad y Especificidad de un modelo en un conjunto de prueba.


Recuerdas qué AIMA te contó una vez:

'La sensibilidad caracteriza la capacidad de la prueba para detectar la enfermedad en sujetos enfermos. La especificidad caracteriza la capacidad de la prueba para detectar la ausencia de la enfermedad en sujetos sanos.'


Implemente la función `seEs` que retorne una tupla de tipo (SE, ES) donde _SE_ es la sensibilidad y _ES_ es la especificidad.

**Peso del punto: 1.0**

In [7]:
from sklearn.metrics import confusion_matrix
def seEs(y_true, y_pred):
    '''
    Entrada:
    y_true: lista con las etiquetas originales del dataset
    y_pred: etiquetas predichas por un modelo
    Salida:
    (SE, ES) : SE es la Sensibilidad, ES la Especificidad
    '''
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    SE = tp / (tp + fn)
    ES = tn / (tn + fp)
    return (SE,ES)